In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy

###Проведемо А/В тестування впровадження фічі 

необхідно взяти одноріну групу. 
беремо цю групу лише тому, що вважаємо її однорідною з точки зору спожицьких звичок, а не тому що вона купувала ноути

In [5]:
laptop=pd.read_csv('laptop.csv')

In [9]:
laptop.head()

,userid,spending,age,visits
0,31,1499,32,12
1,32,799,23,40
2,33,1200,45,22
3,34,0,59,126
4,35,1350,17,85


розділимо групу на 2 підгрупи випадковим чином

In [13]:
np.random.seed(18811015)
laptop.loc[:,'groupassignment']=1*(np.random.random(len(laptop.index))>0.5)
groupA=laptop.loc[laptop['groupassignment']==0,:].copy()
groupB=laptop.loc[laptop['groupassignment']==1,:].copy()

візьмемо отримані через деякий час результати тесту

In [21]:
emailresults=pd.read_csv('emailresults2.csv')

In [25]:
emailresults.head()

,userid,revenue
0,31,100
1,32,0
2,33,50
3,34,800
4,35,75


доэднуєднуємо результати тесту до початкових підгіруп

In [29]:
groupA_withrevenue=groupA.merge(emailresults,on='userid')
groupB_withrevenue=groupB.merge(emailresults,on='userid')

проведемо наше т-тестування і отримаємо п-значення

In [33]:
scipy.stats.ttest_ind(groupA_withrevenue['revenue'],groupB_withrevenue['revenue'])

TtestResult(statistic=-2.381320497676198, pvalue=0.024288828555138562, df=28.0)

п-значення менше порогового 0,05, тому вважаємо результат статистично значимим (відкидаємо нульову гіпотезу про те, що це 
групи з однієї вибірки).
Резульат - фічу можна впроваджувати і/або продовжувати подальше тестування вже з новою фічею